In [1]:
import pandas as pd
import mysql.connector

# Connecting database information
db_config = {
    "host": "localhost",
    "user": "root",    
    "password": "",    
    "database": "pizza_sales"    
}

csv_file_path = "/Users/aghedoemmanuel/Desktop/MySQLstuffs/order_details.csv" 


table_name = "order_details"

try:
    df = pd.read_csv(csv_file_path)
    
    # We rename column 'Site_ID' to 'Station_Site_ID' so as to match with the csv data
    if 'Site_ID' in df.columns:
        df.rename(columns={"Site_ID": "Station_Site_ID"}, inplace=True)
    else:
        print("Column 'Site_ID' not found in the CSV file.")
        exit()
    
    # Remove rows with null values in 'Station_Site_ID'
    if 'Station_Site_ID' in df.columns:
        df.dropna(subset=['Station_Site_ID'], inplace=True)
        print(f"Rows with null 'Station_Site_ID' removed. Remaining rows: {len(df)}")
    else:
        print("Column 'Station_Site_ID' not found in the dataset after renaming.")
        exit()
    
    # Reformat 'Date_Time' to match MySQL datetime format
    if 'Date_Time' in df.columns:
        df['Date_Time'] = pd.to_datetime(df['Date_Time'], errors='coerce')  # Convert to datetime
        df['Date_Time'] = df['Date_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')  # Format as MySQL datetime
        if df['Date_Time'].isnull().any():
            print("Warning: Some 'Date_Time' values could not be converted. Check the data.")
    else:
        print("Column 'Date_Time' not found in the dataset.")
        exit()
except Exception as e:
    print(f"Error reading the CSV file: {e}")
    exit()

try:
    connection = mysql.connector.connect(**db_config)
    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f"Database connection error: {err}")
    exit()


try:
    for _, row in df.iterrows():
        # Create an SQL INSERT query with placeholders
        placeholders = ", ".join(["%s"] * len(df.columns))
        columns = ", ".join(df.columns)
        sql_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        
        # Execute the query with row data
        cursor.execute(sql_query, tuple(row))
    
    # Commit the transaction
    connection.commit()
    print("Data imported successfully.")
except Exception as e:
    connection.rollback()
    print(f"Error inserting data into the database: {e}")
finally:
    cursor.close()
    connection.close()


Error reading the CSV file: 'utf-8' codec can't decode byte 0xb6 in position 10: invalid start byte
Error inserting data into the database: name 'df' is not defined
